# Setup

In [16]:
# Hardware-aware device selection
import torch

def get_device() -> str:
    """
    Returns the optimal device string for running PyTorch models, prioritizing CUDA (NVIDIA GPUs),
    Apple Silicon MPS (Metal), and falling back to CPU. Optionally checks for AMD ROCm/HIP.

    Returns
    ----------
    - device: str
        Name of the device to use: "cuda", "mps", "cpu", or "hip".

    Notes
    ----------
    - On NVIDIA GPUs, "cuda" is used for hardware acceleration.
    - On Apple Silicon, "mps" uses Metal Performance Shaders, providing hardware acceleration.
    - On AMD, "hip" (ROCm) may be available, but this is uncommon.
    - If no GPU is available, it defaults to "cpu".
    Raises
    ----------
    - None
    """
    # Check for CUDA (NVIDIA GPUs)
    if torch.cuda.is_available():
        return "cuda"
    # or MPS (Apple Silicon)
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps"
    # or ROCm (AMD)
    if hasattr(torch.version, "hip") and torch.version.hip is not None:
        return "hip"
    # Fallback to CPU
    return "cpu"

device = get_device()
print(f"Using device: {device}")


Using device: mps


# Building the emotionRegressor

This class loads both transformers, runs inference, and outputs the 35d mood vector for a single input string. Handles device placement and memory cleanup.


In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

class EmotionRegressor:
    """
    Loads and manages two transformer models for emotional analysis, producing a 35-dimensional mood vector for a given text input.
    
    Models:
    ----------
    - SamLowe/roberta-base-go_emotions: Predicts confidence scores for 28 discrete emotions (GoEmotions).
    - j-hartmann/emotion-english-roberta-large: Predicts confidence scores for 7 Ekman emotion categories.

    Parameters
    ----------
    - individual_emotion_model_name: str
        HuggingFace model name for the 28d emotion regressor (default: "SamLowe/roberta-base-go_emotions").
    - ekman_category_model_name: str
        HuggingFace model name for the 7d Ekman regressor (default: "j-hartmann/emotion-english-roberta-large").
    - device: str or None
        Device to use for inference ("cuda", "mps", "cpu", etc). If None, selects best available.

    Methods
    -------
    - predict(text: str) -> np.ndarray
        Returns a 35d mood vector for a given text.
    - cleanup() -> None
        Frees model memory for safe re-initialization.
    """
    def __init__(self,
                 individual_emotion_model_name: str = "SamLowe/roberta-base-go_emotions",
                 ekman_category_model_name: str = "j-hartmann/emotion-english-roberta-large",
                 device: str = None):
        """
        Initialize the EmotionRegressor and load both models on the appropriate device.

        Parameters
        ----------
        - individual_emotion_model_name: str
            Name or path for the 28-emotion model.
        - ekman_category_model_name: str
            Name or path for the 7-Ekman-category model.
        - device: str or None
            Device for inference.

        Returns
        -------
        - None

        Raises
        -------
        - RuntimeError: if model loading fails or device is invalid.
        """
        # Load SamLowe model for individual emotions
        self.device = device or get_device()
        self.individual_emotion_tokenizer = AutoTokenizer.from_pretrained(individual_emotion_model_name)
        self.individual_emotion_model = AutoModelForSequenceClassification.from_pretrained(individual_emotion_model_name).to(self.device)
        self.individual_emotion_model.eval()
        
        # Load hartmann model for Ekman categories
        self.ekman_category_tokenizer = AutoTokenizer.from_pretrained(ekman_category_model_name)
        self.ekman_category_model = AutoModelForSequenceClassification.from_pretrained(ekman_category_model_name).to(self.device)
        self.ekman_category_model.eval()

    def predict(self, text: str) -> np.ndarray:
        """
        Generates a 35-dimensional mood vector for the given input text.

        Parameters
        ----------
        - text: str
            The text to analyze for emotion content.

        Returns
        -------
        - mood_vector: np.ndarray, shape (35,)
            Concatenated vector: [28d GoEmotions scores] + [7d Ekman scores]

        Raises
        -------
        - ValueError: if the text input is empty or not a string.
        - RuntimeError: if inference fails due to hardware or model issues.
        """
        # Validate input
        if not isinstance(text, str) or not text.strip():
            raise ValueError("Input text must be a non-empty string.")
        
        # Lowe model output (28d)
        inputs = self.individual_emotion_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            individual_emotion_logits = self.individual_emotion_model(**inputs).logits
            individual_emotion_probs = torch.sigmoid(individual_emotion_logits).cpu().numpy().flatten()  # (28,)
            
        # Hartmann model output (7d)
        inputs = self.ekman_category_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            ekman_category_logits = self.ekman_category_model(**inputs).logits
            ekman_category_probs = torch.sigmoid(ekman_category_logits).cpu().numpy().flatten()  # (7,)
            
        # Combine into 35d vector
        mood_vector = np.concatenate([individual_emotion_probs, ekman_category_probs])
        return mood_vector

    def cleanup(self) -> None:
        """
        Releases GPU/CPU memory by deleting model weights.

        Returns
        -------
        - None
        """
        # Delete models to free memory
        del self.individual_emotion_model, self.ekman_category_model
        import gc
        # Run garbage collection to free up memory
        gc.collect()
        # Clear CUDA cache if using GPU
        if self.device == "cuda":
            import torch
            torch.cuda.empty_cache()


# Demo of 35-dimensional mood_vector

## Hard-coded example

In [18]:
# Initialize and demo
emotion_regressor = EmotionRegressor(device=device)
sample_text = "I'm exhausted and nothing feels worth it. I just want to sleep and not think about school."
mood_vector = emotion_regressor.predict(sample_text)
print("35d mood_vector:", mood_vector)
print("Shape:", mood_vector.shape)
emotion_regressor.cleanup()


35d mood_vector: [0.00326003 0.00309266 0.00642972 0.06645694 0.01301785 0.00869486
 0.00228629 0.00404051 0.56660634 0.2125662  0.02513171 0.01049308
 0.00295363 0.00560903 0.00179183 0.00208721 0.00450544 0.01198612
 0.00578792 0.00637367 0.0129275  0.00123158 0.01425911 0.00374252
 0.00773545 0.18360835 0.00220227 0.07846191 0.4080965  0.24985977
 0.27866238 0.210173   0.69552344 0.973397   0.29766074]
Shape: (35,)


## Live-user example (doesn't work)

In [19]:
# while True:
#     user_input = input("Enter text (or 'quit'): ")
#     if user_input.lower() == "quit":
#         break
#     mood_vector = emotion_regressor.predict(user_input)
#     print("35d mood_vector:", mood_vector)


## Tinkering with output display format

In [20]:
import pandas as pd

# These should be the order of the GoEmotions and Ekman categories used by your models
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
    'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness',
    'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
EKMAN_LABELS = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

def pretty_print_mood_vector(mood_vector):
    """
    Prints the 35d mood vector in a readable format using pandas DataFrame.
    """
    if len(mood_vector) != 35:
        raise ValueError("Mood vector must be length 35")
    data = {**{f"{e}": v for e, v in zip(GOEMOTIONS_LABELS, mood_vector[:28])},
            **{f"ekman_{e}": v for e, v in zip(EKMAN_LABELS, mood_vector[28:])}}
    df = pd.DataFrame([data])
    print(df.T.rename(columns={0: 'confidence'}))  # .T to flip to (label, value) per row

# Usage
pretty_print_mood_vector(mood_vector)


                confidence
admiration        0.003260
amusement         0.003093
anger             0.006430
annoyance         0.066457
approval          0.013018
caring            0.008695
confusion         0.002286
curiosity         0.004041
desire            0.566606
disappointment    0.212566
disapproval       0.025132
disgust           0.010493
embarrassment     0.002954
excitement        0.005609
fear              0.001792
gratitude         0.002087
grief             0.004505
joy               0.011986
love              0.005788
nervousness       0.006374
optimism          0.012927
pride             0.001232
realization       0.014259
relief            0.003743
remorse           0.007735
sadness           0.183608
surprise          0.002202
neutral           0.078462
ekman_anger       0.408096
ekman_disgust     0.249860
ekman_fear        0.278662
ekman_joy         0.210173
ekman_neutral     0.695523
ekman_sadness     0.973397
ekman_surprise    0.297661


# attempt 2 at live-use

In [21]:
# # Prompt user for input
# user_text = input("Enter a sentence to analyze emotion: ")

# # Preprocess and run the emotion regressor as you would in your pipeline
# # For example:
# # processed = preprocess(user_text)
# # pred = model(processed)
# # But this will depend on your actual pipeline code

# # Replace 'predict_emotion' with whatever function or method you use to predict
# result = predict_emotion(user_text)  # <- your existing function
# print("Emotion scores:", result)


# attempt 3 at live use (single-input)

In [ ]:
emotion_regressor = EmotionRegressor(device=device)
user_input = input("Enter text: ")
mood_vector = emotion_regressor.predict(user_input)


In [26]:

print(user_input)

i MAY have finally figured this out. it probably isn't going to work, it never does. but maybe, just maybe we might make some progress


In [24]:
pretty_print_mood_vector(mood_vector)


                confidence
admiration        0.005581
amusement         0.001743
anger             0.001476
annoyance         0.016229
approval          0.055112
caring            0.005649
confusion         0.078777
curiosity         0.005919
desire            0.075906
disappointment    0.093595
disapproval       0.030504
disgust           0.001945
embarrassment     0.001855
excitement        0.003814
fear              0.003635
gratitude         0.001750
grief             0.000980
joy               0.005191
love              0.002745
nervousness       0.004616
optimism          0.670809
pride             0.001413
realization       0.086809
relief            0.002804
remorse           0.003152
sadness           0.005043
surprise          0.004132
neutral           0.147067
ekman_anger       0.220323
ekman_disgust     0.026956
ekman_fear        0.315277
ekman_joy         0.665933
ekman_neutral     0.892251
ekman_sadness     0.710420
ekman_surprise    0.876877


In [27]:
print(mood_vector)

[0.00558061 0.00174305 0.00147633 0.01622912 0.05511187 0.00564944
 0.07877744 0.00591917 0.07590605 0.09359536 0.03050382 0.0019449
 0.00185502 0.00381371 0.00363517 0.00175045 0.00098024 0.00519138
 0.0027446  0.00461625 0.6708089  0.0014134  0.08680949 0.00280388
 0.00315243 0.00504341 0.00413214 0.14706708 0.22032277 0.02695568
 0.31527734 0.66593313 0.89225143 0.71041965 0.8768775 ]


# attempt 4 at live use (multi-input)

In [28]:
# Only run this once per session! (Don't re-instantiate every input)
# emotion_regressor = EmotionRegressor(device=device)   # Already instantiated above

while True:
    user_input = input("Enter text (or 'quit'): ")
    if user_input.lower() == "quit":
        break
    try:
        mood_vector = emotion_regressor.predict(user_input)
        pretty_print_mood_vector(mood_vector)
    except Exception as e:
        print("Error:", e)

# Optional: Clean up when done
emotion_regressor.cleanup()


                confidence
admiration        0.002158
amusement         0.002357
anger             0.023674
annoyance         0.101695
approval          0.011748
caring            0.004046
confusion         0.004121
curiosity         0.001329
desire            0.001750
disappointment    0.045169
disapproval       0.829912
disgust           0.007823
embarrassment     0.002682
excitement        0.001749
fear              0.001828
gratitude         0.002869
grief             0.000682
joy               0.002302
love              0.001580
nervousness       0.000829
optimism          0.005230
pride             0.000443
realization       0.011552
relief            0.001137
remorse           0.001295
sadness           0.008254
surprise          0.004164
neutral           0.100760
ekman_anger       0.653954
ekman_disgust     0.552545
ekman_fear        0.312908
ekman_joy         0.076213
ekman_neutral     0.410780
ekman_sadness     0.827198
ekman_surprise    0.494735
                confidence
a

In [29]:
print(mood_vector)

[8.1711598e-03 1.1433792e-02 7.2058481e-01 1.5992422e-01 3.1038092e-03
 1.0797601e-03 3.5342218e-03 5.0635021e-03 1.9469517e-03 1.2198194e-02
 8.6276485e-03 2.3778711e-02 3.3267050e-03 1.2342469e-02 4.7174064e-03
 1.2431062e-03 1.4114623e-03 7.4007832e-03 6.6001620e-03 1.0772758e-03
 2.0519628e-03 1.2049049e-03 4.9806908e-03 5.1028928e-04 4.3956586e-04
 5.4115616e-03 4.1645490e-02 6.7463093e-02 5.8091789e-01 4.9606922e-01
 1.6059119e-01 2.5877160e-01 3.8705432e-01 1.4884725e-01 9.6559459e-01]
